## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
import pandas as pd
import time 
import tqdm
from pyspark.sql.types import StructType, StructField, StringType,TimestampType
from pyspark.sql.functions import input_file_name, current_timestamp
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count
from pyspark.sql.functions import desc
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode, lower, regexp_replace,to_timestamp,from_unixtime
from pyspark.sql.types import StringType, ArrayType
import string
import nltk
from nltk.corpus import stopwords
from pyspark.sql.functions import length


## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
files = dbutils.fs.ls(TWEET_SOURCE_PATH)  

In [0]:
print("Number of Files:",len(files))

In [0]:
display(files)

In [0]:
first_file_path = files[0].path

In [0]:
dbutils.fs.head(first_file_path)

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source", input_file_name(), True),
    StructField("processing_time", current_timestamp(), True)
])

raw_data = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "json") \
    .schema(schema) \
    .load(TWEET_SOURCE_PATH)

In [0]:
display(raw_data,streamName="rawData")

In [0]:
# stop_named_stream(spark, "rawData")

In [0]:
dbutils.fs.rm(BRONZE_DELTA, recurse=True)
dbutils.fs.rm(BRONZE_CHECKPOINT, recurse=True)

In [0]:
bronze_stream = raw_data.select(
    "date",
    "user",
    "text",
    "sentiment",
    "processing_time",
    "datasource",
    # input_file_name().alias("datasource"),
    # current_timestamp().alias("processing_time"),
)

In [0]:
(
    bronze_stream.select(
        "date",
        "user",
        "text",
        "sentiment",
        "datasource",
        "processing_time"
    )
    .writeStream.format("delta")
    .outputMode("append")
    .option("checkpointLocation", BRONZE_CHECKPOINT)
    .option("mergeSchema", "true")
    .queryName("bronze_stream")
    .start(BRONZE_DELTA)
)

In [0]:
untilStreamIsReady("bronze_stream")

In [0]:
display(bronze_stream,"bronze_stream_display")

In [0]:
display(bronze_table.history())

In [0]:
for stream in spark.streams.active:
    print(stream.name)

In [0]:
%sql
OPTIMIZE BRONZE_DELTA

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# ENTER YOUR CODE HERE
# # Create a Spark session with Delta support
# spark = SparkSession.builder \
#     .appName("DeltaTableExploration") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .getOrCreate()

# Load the Delta table
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)


In [0]:
total_tweets = bronze_df.count()
print(f"Total number of tweets captured: {total_tweets}")

In [0]:
nan_null_stats = bronze_df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in bronze_df.columns
])
nan_null_stats.show()

In [0]:
tweets_per_user = bronze_df.groupBy("user").count().orderBy(desc("count"))
tweets_per_user.show()

In [0]:
mentions = bronze_df.filter(bronze_df['text'].contains("@")).count()
no_mentions = total_tweets - mentions

print(f"Number of tweets with at least one mention: {mentions}")
print(f"Number of tweets with no mentions: {no_mentions}")

In [0]:
# Convert to Pandas DataFrame for plotting
top_tweeters = tweets_per_user.limit(20).toPandas()

# Plotting
plt.figure(figsize=(10, 8))
plt.bar(top_tweeters['user'], top_tweeters['count'], color='blue')
plt.xlabel('User Handles')
plt.ylabel('Number of Tweets')
plt.title('Top 20 Tweeters')
plt.xticks(rotation=90)
plt.show()


## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

spark = SparkSession.builder.appName("Bronze to Silver Transformation").getOrCreate()

In [0]:
import string

def clean_text(text):
    import nltk
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords
    # Initialize the lemmatizer
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('stopwords')
    lemmatizer = WordNetLemmatizer()
    
    # Get English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Lemmatize words, remove stopwords and extra spaces
    # Tokenize the text, lemmatize each word, and remove stopwords
    words = nltk.word_tokenize(text)
    lemmatized_text = ' '.join([lemmatizer.lemmatize(word).lower() for word in words if word.lower() not in stop_words])
    
    return lemmatized_text

clean_text_udf = udf(clean_text, StringType())


In [0]:
def extract_mentions(text):
    # Find all @mentions and remove '@'
    mentions = ' '.join(set(part[1:] for part in text.split() if part.startswith('@')))
    return mentions

extract_mentions_udf = udf(extract_mentions, StringType())

In [0]:
dbutils.fs.rm(SILVER_DELTA, recurse=True)
dbutils.fs.rm(SILVER_CHECKPOINT, recurse=True)

In [0]:
bronze_df = spark.readStream \
    .format("delta") \
    .option("ignoreChanges", "true") \
    .load(BRONZE_DELTA)


In [0]:
# spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# silver_df = bronze_df.withColumn("mention", extract_mentions_udf(col("text"))) \
#     .filter(length(col("mention")) > 0) \
#     .withColumn("text", regexp_replace(col("text"), r'@\w+', '')) \
#     .withColumn("cleaned_text", clean_text_udf(col("text"))) \
#     .withColumn("timestamp", to_timestamp("date", "EEE MMM dd HH:mm:ss zzz yyyy")) \
#     .select(
#         col("timestamp"),
#         col("mention"),
#         col("cleaned_text"),
#         col("sentiment")
#     )

In [0]:
def remove_mentions_from_text(cleaned_text, mention):
    if mention and mention.strip():
        return cleaned_text.replace(mention, "")
    return cleaned_text

# Register the function as a UDF
remove_mentions_udf = udf(remove_mentions_from_text, StringType())

In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
silver_df = bronze_df.withColumn("mention", extract_mentions_udf(col("text"))) \
    .filter(length(col("mention")) > 0) \
    .withColumn("cleaned_text", clean_text_udf(col("text"))) \
    .withColumn("timestamp", to_timestamp("date", "EEE MMM dd HH:mm:ss zzz yyyy")) \
    .select(
        col("timestamp"),
        col("mention"),
        col("cleaned_text"),
        col("sentiment")
    )\
    .withColumn("cleaned_text",remove_mentions_udf(lower(col("cleaned_text")), lower(col("mention"))))

In [0]:
(silver_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", SILVER_CHECKPOINT)
    .start(SILVER_DELTA))

In [0]:
display(silver_df,streamName='SilverStreamDisplay')

In [0]:
silver_delta = spark.read.format('delta').load(SILVER_DELTA)

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# ENTER YOUR CODE HERE
import mlflow
from mlflow.pyfunc import PyFuncModel
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType, StructType, StructField,IntegerType
import pandas

In [0]:
# Assuming the model is registered under the name 'HF_TWEET_SENTIMENT'
model_name = "HF_TWEET_SENTIMENT"

# Load the model from MLflow
model_uri = f"models:/{model_name}/Production"
sentiment_model = mlflow.pyfunc.load_model(model_uri)

In [0]:
dbutils.fs.rm(GOLD_DELTA, recurse=True)
dbutils.fs.rm(GOLD_CHECKPOINT, recurse=True)

In [0]:
from pyspark.sql.functions import pandas_udf, PandasUDFType, col, when
import pandas as pd
import mlflow.pyfunc
 
# Assuming the model is loaded correctly and available as sentiment_model
# Load the model outside the UDF, globally if possible to avoid reloading per task
#sentiment_model = mlflow.pyfunc.load_model("models:/your_model_name/Production")
 
@pandas_udf("struct<sentiment_label:string, sentiment_score:double, sentiment_id:int>", PandasUDFType.SCALAR)
def predict_sentiment(texts: pd.Series) -> pd.DataFrame:
    # Apply prediction to each text entry
    results = texts.apply(lambda text: sentiment_model.predict([text]))
 
    # Extract data into separate series
    labels = results.apply(lambda result: result['label'][0])
    scores = results.apply(lambda result: result['score'][0])
    sentiment_ids = labels.apply(lambda label: 1 if label == 'POS' else 0)
 
    return pd.DataFrame({
        'sentiment_label': labels,
        'sentiment_score': scores,
        'sentiment_id': sentiment_ids
    })
 


In [0]:
silver_delta = spark.read.format("delta").load(SILVER_DELTA)

In [0]:
silver_delta = spark.readStream \
.format("delta") \
.option("ignoreChanges", "true") \
.load(SILVER_DELTA)

In [0]:
gold_df = silver_delta.withColumn("sentiment_result", predict_sentiment(col("cleaned_text")))
gold_df = gold_df.select(
    col("timestamp"),
    col("mention"),
    col("cleaned_text"),
    col("sentiment"),
    col("sentiment_result.sentiment_score").alias("predicted_score"),
    col("sentiment_result.sentiment_label").alias("predicted_sentiment"),
    col("sentiment_result.sentiment_id").alias("predicted_sentiment_id")
)\
.withColumn("sentiment_id", 
                when(col("sentiment") == "positive", 1).otherwise(0))                 


In [0]:
gold_query = gold_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", GOLD_CHECKPOINT) \
    .option("path", GOLD_DELTA) \
    .queryName("gold_stream") \
    .trigger(processingTime='1 milliseconds') \
    .start()

gold_query.awaitTermination()


In [0]:
display(gold_df)

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
import mlflow
from mlflow import log_metric, log_param, log_artifact
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
# ENTER YOUR CODE HERE
gold_df_table = spark.read.format('delta').load(GOLD_DELTA)

In [0]:
print(gold_df_table.count())

In [0]:
# ENTER YOUR CODE HERE

# mlflow.set_experiment('/Shared/Exp3_09-05-2024')

In [0]:
mlflow.end_run()

In [0]:
import mlflow
from delta.tables import DeltaTable
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming `gold_df` is available and ready
#gold_df_plot = spark.read.format("delta").load(GOLD_DELTA)  # Replace with your table's name if different

# Start an MLflow run
mlflow.set_experiment('/Shared/Exp6_11-05-2024')
with mlflow.start_run():
    # Convert the necessary columns to pandas for metric calculation
    pandas_df = updated_gold_df.select("sentiment_id", "predicted_sentiment_id").toPandas()
    y_true = pandas_df['sentiment_id']
    y_pred = pandas_df['predicted_sentiment_id']

    # Calculate metrics
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    
    # Log metrics
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Generate and save confusion matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.savefig("confusion_matrix.png")
    plt.close()

    # Log the confusion matrix as an MLflow artifact
    mlflow.log_artifact("confusion_matrix.png")

    # Log model and version details
    mlflow.log_param("model_name", "HF_TWEET_SENTIMENT")
    mlflow.log_param("mlflow_version", mlflow.__version__)

    # Log the version of the Delta Table
    delta_table = DeltaTable.forPath(spark, SILVER_DELTA)
    version = delta_table.history(1).select("version").collect()[0]["version"]
    mlflow.log_param("delta_table_version", version)


## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
from pyspark.sql.functions import explode, split, col

# Assuming updated_gold_df is loaded and available
updated_gold_df = gold_df_table.withColumn(
    "predicted_sentiment_id",
    when(gold_df_table["predicted_sentiment"] != "NEG", 1).otherwise(0)
)

# Show updated DataFrame or write it back to Delta if needed
updated_gold_df.show()

# Split the 'mention' string into an array of mentions, then explode it into multiple rows
expanded_df = updated_gold_df.withColumn("mention", explode(split(col("mention"), " ")))

# Filter out empty mentions
filtered_df = expanded_df.filter(expanded_df["mention"] != "")

# Group by 'mention' and pivot on 'predicted_sentiment' to count occurrences of each sentiment
result_df = filtered_df.groupBy("mention").pivot("predicted_sentiment", ["POS", "NEU", "NEG"]).count()

# Calculate the total mentions for each user
result_df = result_df.withColumn("total_mentions", col("POS") + col("NEU") + col("NEG"))

# Order by total_mentions in descending order
result_df = result_df.orderBy(col("total_mentions").desc())

# Show result or continue processing
result_df.show()



In [0]:
# save_path = "/tmp/labuser104917-3110160/gold_aggregation_delta"

# # Write the DataFrame to the specified Delta location
# result_df.write.format("delta").mode("overwrite").save(save_path)

In [0]:
# streamingDF = spark.readStream \
#     .format("delta") \
#     .load(save_path)

In [0]:
dbutils.widgets.dropdown("Update Visualization", "OFF", ["ON", "OFF"])

In [0]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# def plot_sentiment(data, sentiment_type, color):
#     top_mentions = data.orderBy(col(sentiment_type).desc()).limit(20)
#     pd_df = top_mentions.toPandas()
    
#     plt.figure(figsize=(12, 8))
#     sns.barplot(x=pd_df['mention'], y=pd_df[sentiment_type], color=color)
#     plt.xticks(rotation=45, ha='right')
#     plt.title(f'Top 20 Mentions by {sentiment_type} Sentiment')
#     plt.xlabel('Mention')
#     plt.ylabel('Counts')
#     plt.tight_layout()
#     plt.show()

# def process_and_plot(batch_df, batch_id):
#     # Convert batch DataFrame to a static DataFrame
#     data = spark.sql("SELECT * FROM aggregated_data")
#     plot_sentiment(data, "POS", "green")
#     plot_sentiment(data, "NEG", "red")


In [0]:
# from pyspark.sql import functions as F
# query = (streamingDF.writeStream 
#     .foreachBatch(process_and_plot)
#     .queryName("aggregated_data") 
#     .outputMode("update") 
#     .trigger(processingTime='1 millisecond')
#     .start())

# # plot_query = result_df.writeStream \
# #     .foreachBatch(process_and_plot) \
# #     .outputMode("complete") \
# #     .start()



# # query = (gold_df.writeStream
# #          .foreachBatch(process_and_plot)
# #          .outputMode("complete")
# #          .option("checkpointLocation", "/path/to/checkpoint")
# #          .trigger(processingTime='1 minute')  # Set a reasonable trigger to refresh the plots
# #          .start())


In [0]:
# from pyspark.sql.functions import col, sum
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Assuming save_path is correctly set to your Delta table's path
# streamingDF = spark.readStream.format("delta").load(save_path)

# def process_and_plot(batch_df, batch_id):
#     if dbutils.widgets.get("Update Visualization") == "ON":
#         # Perform necessary transformations
#         data = batch_df.groupBy("mention").agg(
#             sum(col("POS")).alias("POS"),
#             sum(col("NEG")).alias("NEG")
#         ).withColumn("total_mentions", col("POS") + col("NEG")).orderBy(col("total_mentions").desc()).limit(20)

#         # Convert to pandas DataFrame for plotting
#         pd_df_pos = data.select("mention", "POS").toPandas()
#         pd_df_neg = data.select("mention", "NEG").toPandas()

#         # Plotting
#         plt.figure(figsize=(12, 8))
#         plt.subplot(2, 1, 1)
#         sns.barplot(x='mention', y='POS', data=pd_df_pos, color='green')
#         plt.title('Top 20 Mentions by Positive Sentiment')
#         plt.xticks(rotation=45, ha='right')

#         plt.subplot(2, 1, 2)
#         sns.barplot(x='mention', y='NEG', data=pd_df_neg, color='red')
#         plt.title('Top 20 Mentions by Negative Sentiment')
#         plt.xticks(rotation=45, ha='right')

#         plt.tight_layout()
#         plt.show()

# # Streaming query setup
# query = streamingDF.writeStream \
#     .foreachBatch(process_and_plot) \
#     .outputMode("update") \
#     .trigger(processingTime='1 second') \
#     .start()


In [0]:
result_pd = result_df.toPandas()

In [0]:
import matplotlib.pyplot as plt

# Plot for positive sentiments
top_positive = result_pd.nlargest(20, 'POS')
plt.figure(figsize=(10, 8))
plt.barh(top_positive['mention'], top_positive['POS'], color='green')
plt.xlabel('Count of Positive Mentions')
plt.ylabel('Mentions')
plt.title('Top 20 Mentions with Positive Sentiment')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest value at the top
plt.show()


In [0]:
# Plot for negative sentiments
top_negative = result_pd.nlargest(20, 'NEG')
plt.figure(figsize=(10, 8))
plt.barh(top_negative['mention'], top_negative['NEG'], color='red')
plt.xlabel('Count of Negative Mentions')
plt.ylabel('Mentions')
plt.title('Top 20 Mentions with Negative Sentiment')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest value at the top
plt.show()


## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# ENTER YOUR CODE HERE

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE